### Load modules

In [1]:
import pandas as pd
import numpy as np
import os
# sub_dir = '/Users/dominicbates/Documents/GitHub/app-review-classifier/'
# os.chdir(sub_dir)
# import app_review_classifier.review_scraper as scraper


### My Module

In [ ]:
import pandas as pd
import numpy as np
from app_store_scraper import AppStore
from google_play_scraper import reviews_all, reviews, Sort, app
import time


def get_apple_reviews(app_name, app_id, country):
    
    # Try and get scrape reviews...
    for n in range(10):
        reviews = AppStore(country = country, app_name = app_name, app_id = app_id)
        #reviews.review(how_many=100000)
        reviews.review()
        reviews_dict = reviews.reviews
        del reviews
        
        # If no results, retry after a few seconds
        if len(reviews_dict) < 2:
            print("No results found for",country,"- let's wait a sec, then retry...")
            time.sleep(10+(n*2)) # Increase wait each time up to 30s
        else:
            print('Found some results for '+country+'! Moving on...')
            break

    if len(reviews_dict) < 2:
        print('ERROR: Tried 20 times and didn\'t get proper response')
            
    return reviews_dict



def apple_reviews_to_df(reviews_dict, app_name):
        # Turn to dataframe
    col_date = []
    col_review = []
    col_title = []
    col_rating = []
    col_app = []
    col_unique_id = []
    for review in reviews_dict:
        col_date.append(review['date'])
        col_title.append(review['title'])
        col_review.append(review['review'])
        col_rating.append(review['rating'])
        col_unique_id.append('apple_'+app_name+'_'+review['userName'])
        col_app.append(app_name)
    
    reviews_df = pd.DataFrame({'date':col_date,
                               'title':col_title,
                               'review':col_review,
                               'rating':col_rating,
                               'app':col_app,
                               'uniqueid':col_unique_id})
    return reviews_df


def get_google_reviews(app_id, country, lang):
    reviews_dict = reviews_all(app_id,
                               sleep_milliseconds=0, # defaults to 0
                               country=country, # Changing this doesn't do anything - 3078 reviews show up regardless of what country code - this is probably all EN reviews, after checking how many per country with "app()"
                               lang=lang) # Just choosing EN filters out some english reviews, e.g. FR is mostly english)
    return reviews_dict


def google_reviews_to_df(reviews_dict, app_name):
        # Turn to dataframe
    col_date = []
    col_review = []
    col_title = []
    col_rating = []
    col_app = []
    col_unique_id = []
    for review in reviews_dict:
        col_date.append(review['at'])
        col_title.append('')
        col_review.append(review['content'])
        col_rating.append(review['score'])
        col_unique_id.append('google_'+app_name+'_'+review['userName'])
        col_app.append(app_name)
    
    reviews_df = pd.DataFrame({'date':col_date,
                               'title':col_title,
                               'review':col_review,
                               'rating':col_rating,
                               'app':col_app,
                               'uniqueid':col_unique_id})
    return reviews_df
    


def get_all_reviews(country_list = None):

    # Load default countries
    if country_list is None:
        countries_english = ['US','GB','CA','AU','ZA','IE','NZ'] # English speaking countries
        countries_other = ['IN','ID','BR','DE','ES','NL','SE','SG'] # Populous countries where vast majority of reviews are in english
        countries_list = countries_english + countries_other

    # Scrape apple reviews
    apple_reviews_economist = {}
    apple_reviews_espresso = {}
    print('Getting apple reviews...')
    for country in countries_list: 
        apple_reviews_economist[country] = get_apple_reviews('the-economist', app_id='1239397626', country=country)
        apple_reviews_espresso[country] = get_apple_reviews('espresso-from-the-economist', app_id='896628003', country=country)
    print('Apple reviews downloaded!')

    # Scrape google reviews
    google_reviews_economist = {}
    google_reviews_espresso = {}
    print('Getting google reviews...')
    google_reviews_economist['All'] = get_google_reviews('com.economist.lamarr', country='US', lang='EN') # I think this is is all english reviews worldwide
    google_reviews_espresso['All'] = get_google_reviews('com.economist.darwin', country='US', lang='EN') # I think this is is all english reviews worldwide
    print('Google reviews downloaded!')

    # Turn to dataframe and combine
    all_reviews_list = []

    for r in apple_reviews_economist:
        all_reviews_list.append(apple_reviews_to_df(apple_reviews_economist[r], 'The Economist (Apple)'))
    for r in apple_reviews_espresso:
        all_reviews_list.append(apple_reviews_to_df(apple_reviews_espresso[r], 'Espresso (Apple)'))
    for r in google_reviews_economist:
        all_reviews_list.append(google_reviews_to_df(google_reviews_economist[r], 'The Economist (Google)'))
    for r in google_reviews_economist:
        all_reviews_list.append(google_reviews_to_df(google_reviews_espresso[r], 'Espresso (Google)'))

    final_reviews_df = pd.concat(all_reviews_list).reset_index(drop=True)

    return final_reviews_df





### Run get reviews and save

In [ ]:
reviews = get_all_reviews()


In [ ]:
reviews.to_parquet('/Users/dominicbates/Documents/GitHub/app-review-classifier/data/training_sample/raw_review_data.parquet')
